# Install Wikibase
During the workshop at the [National institute of Informatics (NII)](https://wikibase.peatix.com/) you will be guided through the step required to install a local Wikibase instance. Either on your local computer, a server, or a VM. Wikibase is the underlying infrastructure of [Wikidata](http://www.wikidata.org). Since 2017, wikibase is also available through Docker. The docker image was presented to the wikidata community in 2017 as a birthday present during the first edition of WikidataCon, which was organised to comemorate the [5th anniversary of Wikidata](https://www.wikidata.org/wiki/Wikidata:Fifth_Birthday/Presents), by [addshore](https://www.wikidata.org/wiki/User:Addshore). 

In this workshop we will build on this birthday present. To continue [docker](https://www.docker.com/) and [docker-compose](https://docs.docker.com/compose/install/) needs to be installed. The instructions below are for an install on Ubuntu. For other systems, please consult the instruction page on the respective websites (docker and docker compose).

## Preparing system

### Update the system
```bash 
sudo apt-get update
```

### Install packages to allow apt to use a repository over HTTPS:
```bash
sudo apt-get install --yes \
     apt-transport-https \
     ca-certificates \
     curl \
     gnupg2 \
     software-properties-common
```
     
### Add Docker’s official GPG key:

```bash
curl -fsSL https://download.docker.com/linux/$(. /etc/os-release; echo "$ID")/gpg | sudo apt-key add -
```

### Add Docker's stable repository
```bash
sudo add-apt-repository \
   "deb [arch=amd64] https://download.docker.com/linux/$(. /etc/os-release; echo "$ID") \
   $(lsb_release -cs) \
   stable"
```

### Update the apt package index:
```bash 
sudo apt-get update
```

### Install the latest version of Docker CE
```bash 
sudo apt-get --yes install docker-ce
```

### Run Hellow World
```bash
sudo docker run hello-world
```

##    Install Docker Compose

### Install pip
```bash
sudo apt-get install --yes \
     python-pip
```


### Install docker-composer using pip
```bash
sudo pip install docker-compose
```

##    Clone wikibase-docker    

### Install git
```bash
sudo apt-get install --yes \
     git
```

### Clone the repo
```bash
git clone https://github.com/wmde/wikibase-docker.git
```

### Switch to the wikibase-docker directory
```bash 
cd wikibase-docker/
```

## Run it

### Pull the images from docker hub
```bash
sudo docker-compose pull
```

### Run the services
```bash
sudo docker-compose up --no-build -d
```

## Running it all in one. 
The above steps can also be run by running the [setup.sh](https://github.com/wmde/wikibase-docker/blob/master/setup.sh)

A freshly installed Wikibase is now running on your system and the following screen should be visible

![alt text](img/wikibase_hello_world.png)
![wbqs](img/wdqs_hello_world.png)

Upon completion, you should have the following entry points into your installed wikibase:
* A Wikibase, available at http://127.0.0.1:8181 (or IP address)
* A Wikibase Query Service available at http://127.0.0.1:8282 (or IP address)
* QuickStatements, available at http://127.0.0.1:9191 (or IP address)
* Bare SPARQL (Blazegraph) endpoint at http://127.0.0.1:8989/bigdata/ (or IP address)


# configuring wikibase
Above is the default screen, however it is still a rather empty system. In this step we will be creating properties in the freshly installed Wikibase system. There are two ways to do this. One through the webinterface, the other using the API through a programming library such as the [Wikidata Integrator](https://github.com/SuLab/WikidataIntegrator)

## Manually creating Properties and Items.
Adding Properties and Items is a bit hidden on the installed system. To add either new properties or items, go to the "special pages" in the tools sub-menu in the left banner.

We start creating properties, by selecting "Create a new Property", at the bottom end of those special pages. 

## Programmatically creating Property and Items. 
Creating properties can be a tedious process, especially if the underlying envisioned schema includes many levels of arcs, or an extended number of properties. In that can it can be helpful to create the properties automatically. The following script would create a new property 


In [3]:
from wikidataintegrator import wdi_core, wdi_login


mediawiki_api_url = "http://127.0.0.1:7171/w/api.php"
sparql_endpoint_url = "http://127.0.0.1:7272/proxy/wdqs/bigdata/namespace/wdq/sparql"
username = "Test"
password = "test2"
login = wdi_login.WDLogin(username, password, mediawiki_api_url=mediawiki_api_url)

def createProperty(login, label, description, property_datatype):
  s = []
  localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(mediawiki_api_url,sparql_endpoint_url)
  item = localEntityEngine(data=s)
  item.set_label(label)
  item.set_description(description)
  print(item.write(login, entity_type="property", property_datatype=property_datatype))


## Properties accorind to: https://docs.google.com/spreadsheets/d/1RhxhzRW_Id10uOXiaFeubIG0z6b0I7k9VN6Sg2KqZG8/edit#gid=0
# Session
createProperty(login, "instance of", "is a", "wikibase-item")
createProperty(login, "subclass of", "subclass of", "wikibase-item")
createProperty(login, "same as", "external IRI mapping", "URL")

http://127.0.0.1:7171/w/api.php
Successfully logged in as Test
Error while writing to Wikidata


WDApiError: {'error': {'code': 'failed-save', 'info': 'The save has failed.', 'messages': [{'name': 'wikibase-api-failed-save', 'parameters': [], 'html': {'*': 'The save has failed.'}}, {'name': 'wikibase-validator-label-conflict', 'parameters': ['instance of', 'en', '[[Property:P1|P1]]'], 'html': {'*': '言語コード en に関連付けられたラベル「instance of」を持つプロパティ (<a href="/wiki/Property:P1" title="Property:P1">P1</a>) が既にあります。'}}], '*': 'See http://127.0.0.1:7171/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}}

The following code snippet adds Item to the Wikibase

In [4]:
wdPage = wdi_core.WDItemEngine(new_item=True, mediawiki_api_url=mediawiki_api_url, sparql_endpoint_url=sparql_endpoint_url)
wdPage.set_label("first label", lang='en')
wdPage.set_label("first description", lang="en")
print(wdPage.get_wd_json_representation())

{'claims': {}, 'labels': {'en': {'language': 'en', 'value': 'first description'}}}


/usr/local/lib/python3.7/site-packages/wikidataintegrator/wdi_core.py:168: UserWarning: mapping relation types are being ignored
  warnings.warn("mapping relation types are being ignored")


In [5]:
wdPage.write(login)

'Q1'

In [6]:
wdPage.set_label("eerste label", lang='nl')
wdPage.set_label("eerste beschrijving", lang="nl")

In [7]:
wdPage.write(login)

'Q1'

In [8]:
wdPage2 = wdi_core.WDItemEngine(new_item=True, mediawiki_api_url=mediawiki_api_url, sparql_endpoint_url=sparql_endpoint_url)
wdPage2.set_label("label", lang='en')
wdPage2.set_label("description", lang="en")
wdPage2.write(login)

/usr/local/lib/python3.7/site-packages/wikidataintegrator/wdi_core.py:168: UserWarning: mapping relation types are being ignored
  warnings.warn("mapping relation types are being ignored")


'Q2'

In [11]:
data = [wdi_core.WDItemID(value="Q2", prop_nr='P1')]

In [12]:
wdPage = wdi_core.WDItemEngine(wd_item_id="Q1", data=data, mediawiki_api_url=mediawiki_api_url, sparql_endpoint_url=sparql_endpoint_url)
wdPage.write(login)

/usr/local/lib/python3.7/site-packages/wikidataintegrator/wdi_core.py:168: UserWarning: mapping relation types are being ignored
  warnings.warn("mapping relation types are being ignored")


'Q1'